In [1]:
import xarray as xr
import plotly.express as px
from pathlib import Path
from datetime import datetime

In [2]:
station_name = "PAPA"
release_folder = "../../../../release/"

In [3]:
papa_data = xr.load_dataset("../../2_processed/papa_zooplankton.nc")
papa_data

<xarray.Dataset> Size: 249MB
Dimensions:           (time: 623, is_day: 2, latitude: 12, longitude: 26,
                       depth: 8)
Coordinates:
  * time              (time) datetime64[ns] 5kB 1995-09-24 ... 2020-08-31
  * is_day            (is_day) bool 2B False True
  * latitude          (latitude) float64 96B 46.5 47.5 48.5 ... 55.5 56.5 57.5
  * longitude         (longitude) float64 208B -156.5 -153.5 ... -129.5 -128.5
  * depth             (depth) int64 64B 50 100 150 200 250 300 350 400
Data variables:
    mesh_size         (time, is_day, latitude, longitude, depth) float64 25MB ...
    volume_filtered   (time, is_day, latitude, longitude, depth) float64 25MB ...
    benthos           (time, is_day, latitude, longitude, depth) float64 25MB ...
    crustacean        (time, is_day, latitude, longitude, depth) float64 25MB ...
    chaetognatha      (time, is_day, latitude, longitude, depth) float64 25MB ...
    small_gelatinous  (time, is_day, latitude, longitude, depth) float64 25MB ...
    larvacean         (time, is_day, latitude, longitude, depth) float64 25MB ...
    thaliacea         (time, is_day, latitude, longitude, depth) float64 25MB ...
    others            (time, is_day, latitude, longitude, depth) float64 25MB ...
    total             (time, is_day, latitude, longitude, depth) float64 25MB ...

In [4]:
results = xr.Dataset(
    {
        "day": papa_data["total"].sel(is_day=True).mean(["depth"]).drop_vars("is_day"),
        "night": papa_data["total"]
        .sel(is_day=False)
        .mean(["depth"])
        .drop_vars("is_day"),
    }
)
results

<xarray.Dataset> Size: 3MB
Dimensions:    (time: 623, latitude: 12, longitude: 26)
Coordinates:
  * time       (time) datetime64[ns] 5kB 1995-09-24 1995-09-25 ... 2020-08-31
  * latitude   (latitude) float64 96B 46.5 47.5 48.5 49.5 ... 55.5 56.5 57.5
  * longitude  (longitude) float64 208B -156.5 -153.5 -152.5 ... -129.5 -128.5
Data variables:
    day        (time, latitude, longitude) float64 2MB nan nan nan ... nan nan
    night      (time, latitude, longitude) float64 2MB nan nan nan ... nan nan

In [5]:
day_df = (
    results["day"].mean(["latitude", "longitude"]).to_dataframe().reset_index().dropna()
)
night_df = (
    results["night"]
    .mean(["latitude", "longitude"])
    .to_dataframe()
    .reset_index()
    .dropna()
)


fig = px.line(
    title="Day and Night Series", labels={"value": "Abundance", "time": "Time"}
)

fig.add_scatter(x=day_df["time"], y=day_df["day"], mode="lines", name="Day")

fig.add_scatter(x=night_df["time"], y=night_df["night"], mode="lines", name="Night")

fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Wet weight mg/m3",
    legend_title="Day/Night",
    legend=dict(orientation="h"),
)

fig.show()

In [6]:
data_export = (
    xr.Dataset(
        {
            "day": papa_data["total"]
            .sel(is_day=True)
            .mean(["depth"])
            .drop_vars("is_day"),
            "night": papa_data["total"]
            .sel(is_day=False)
            .mean(["depth"])
            .drop_vars("is_day"),
        }
    )
    .expand_dims({"layer": [0]})
    .transpose("time", "latitude", "longitude", "layer")
)
data_export["day"].attrs = {
    "description": "Zooplankton dry weight by cube meter of sea water at day",
    "units": "mg/m3",
}
data_export["night"].attrs = {
    "description": "Zooplankton dry weight by cube meter of sea water at night",
    "units": "mg/m3",
}
data_export["layer"].attrs = {
    "flag_values": [0, 1, 2],
    "flag_meanings": "epipelagic, upper-mesopelagic, lower-mesopelagic",
    "desc": "The layer depth according to Seapodym definition",
    "standard_name": "layer",
    "long_name": "Layer depth",
    "axis": "Z",
}
data_export.attrs = {
    "station": station_name,
    "creation date": datetime.now().isoformat(),
}
data_export

<xarray.Dataset> Size: 3MB
Dimensions:    (layer: 1, time: 623, latitude: 12, longitude: 26)
Coordinates:
  * layer      (layer) int64 8B 0
  * time       (time) datetime64[ns] 5kB 1995-09-24 1995-09-25 ... 2020-08-31
  * latitude   (latitude) float64 96B 46.5 47.5 48.5 49.5 ... 55.5 56.5 57.5
  * longitude  (longitude) float64 208B -156.5 -153.5 -152.5 ... -129.5 -128.5
Data variables:
    day        (time, latitude, longitude, layer) float64 2MB nan nan ... nan
    night      (time, latitude, longitude, layer) float64 2MB nan nan ... nan
Attributes:
    station:        PAPA
    creation date:  2025-07-09T13:28:41.839086

In [7]:
data_export = data_export.chunk(
    {"time": -1, "latitude": 5, "longitude": 5, "layer": -1}
)

In [8]:
data_export.to_zarr(Path(release_folder) / f"{station_name}_zooplankton.zarr", mode="w")